In [2]:
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist
from nltk import word_tokenize
tt = TweetTokenizer()
from string import punctuation
import numpy as np
matplotlib.rcParams.update({'figure.autolayout': True})

In [4]:
df = pd.read_csv('tablita.csv')

In [5]:
b = 's.'

In [6]:
df['historia_clinica'] = df['historia_clinica'].str.lower().str.replace('(?:\\\\s|^)'+b+'(?:\\\\s|$)',' sindrome ')

In [7]:
df['historia_clinica'] = df['historia_clinica'].str.replace(',',' ')

In [9]:
df['historia_clinica'] = df['historia_clinica'].str.lower().str.replace('(?:\\\\s|^)'+b+'(?:\\\\s|$)','sindrome')
df['historia_clinica'] = df['historia_clinica'].str.replace('.',' ')

In [11]:
df['historia_clinica'] = df['historia_clinica'].str.lower().str.replace('años','anos')
df['historia_clinica'] = df['historia_clinica'].str.lower().str.replace('mama','seno')

In [13]:
stops_manunales = ['paciente','Gen','gen','secuenciacion','molecular','tipo']
pat = r'\\b(?:{})\\b'.format('|'.join(stops_manunales))